In [ ]:
// run this cell to prevent Jupyter from displaying the null output cell
com.twosigma.beakerx.kernel.Kernel.showNullExecutionResult = false;

We need the `Counter` class in this notebook. Run the next cell to compile the class.

In [ ]:
public class Counter {

    protected int value; 
    
    public Counter() {
        this.value = 0;
    }
    
    public Counter(int value) {
        this.value = value;
    }
    
    public Counter(Counter other) {
        this.value = other.value;
    }
    
    public int value() {
        return this.value;
    }
    
    public void advance() {
        if (this.value != Integer.MAX_VALUE) {
            this.value++;
        }
        else {
            this.value = 0;
        }
    }
    
    @Override
    public boolean equals(Object obj) {
        if (this == obj) {
            return true;
        }
        if (!(obj instanceof Counter)) {
            return false;
        }
        Counter other = (Counter) obj;
        if (this.value == other.value) {
            return true;
        }
        return false;
    }
    
    @Override
    public int hashCode() {
        int result = Integer.hashCode(this.value);

        return result;
    }
    
    @Override
    public String toString() {
        return "count: " + this.value;
    }
}

<a id="notebook_id"></a>
# Substitutability

Inheritance models the is-a or is-substitutable-for relationship. What this means is that a subtype object should always be usable whenever a supertype object is required, and doing so will not cause a program to behave unexpectedly.

In object-oriented programming, the term *substitutability* often refers to the Liskov substitution principle (named after [Barbara Liskov](https://en.wikipedia.org/wiki/Barbara_Liskov) who first described the principle). The formal definition of the principle is

> Let $\phi(x)$ be a property provable about objects $x$ of type T. Then $\phi(y)$ should be true for objects $y$ of type S where S is a subtype of T. 

In practice, Liskov substitutability means that subclass objects must behave the same as objects of their supertype. Object behaviour is determined by the object's methods; thus ensuring substitutability requires paying careful attention when overriding methods in a subclass.

## Class invariants

Recall that class invariant is some condition that is true about the state of an object. If a superclass has one or more class invariants then the subclass must also provide those same invariants for Liskov substitutability to hold.

The `Counter` class has the invariant that the value of a counter is always greater than or equal to zero. One way to express this invariant is the following: For any `Counter` object `c` the value of `c.value() >= 0` is `true`. A user of the `Counter` class rightfully can assume that the invariant will old for any `Counter` object. For example, a programmer can write the following method:

In [ ]:
public class InvariantExample {
    
    /**
     * Returns true if the specified counter has an odd value.
     *
     * @return true if the specified counter has an odd value
     */
    public static boolean isOdd(Counter c) {
        return c.value() % 2 == 1;
    }
}

For every possible `Counter` value the above method returns the correct answer (try it for yourself). Now, consider what happens if a programmer creates the following subclass:

In [ ]:
public class DescendingCounter extends Counter {

    public DescendingCounter() {
        super();
    }
    
    public DescendingCounter(int value) {
        super(value);
    }
    
    @Override
    public void advance() {
        this.value--;
    }
}

A `DescendingCounter` is simply a counter that counts down (toward negative infinity) that wraps around to `Integer.MAX_VALUE` when it reaches `Integer.MIN_VALUE`.

**Exercise 1** Test that the `DescendingCounter` class compiles and functions as expected (e.g., `advance` causes the count to decrease by 1, `equals` can be used to compare a `Counter` and `DescendingCounter` for equality, etc).

The method `isOdd(Counter)` now fails when passed an instance of a `DescendingCounter` having a negative value:

In [ ]:
DescendingCounter c = new DescendingCounter();
c.advance();
System.out.println(c.value());    // -1
System.out.println(InvariantExample.isOdd(c));    // -1 is not odd!

**Exercise 2** It is easy to fix the implementation of `isOdd` so that it works with `DescendingCounter` objects. Suggest such a fix.

The above example is a very simple example where a subclass that does not support its superclass invariant will cause code that relies on the invariant to fail. The example illustrates a very important point: The creator of the method `isOdd` can test their code with every expected `Counter` value *and their code will pass all of the tests*. The method does not fail until a subclass is used *that is not fully substitutable for its superclass*.

## Method parameters

Many methods place restrictions on their parameter values. 

An overridden method of a subclass needs to accept *at least* the same range of input parameter values as the method of the superclass, and may accept a wider range. This means that a subclass must not add additional constraints on the set of values that a method accepts as valid parameters. If a subclass adds additional constraints then code created using the superclass contracts will fail when given a subclass object.

Consider the following two classes. A `Shape` is the superclass of two-dimensional shapes where every shape has a positive width and height. The `Shape` class has fields that store the width and height of the shape. A `Rectangle` is a shape that represents rectangles. We may assume that there are other shape classes representing various other two-dimensionsal shapes.

In [ ]:
/**
 * Superclass for shapes that have an independent width and height.
 * The width and height have a minimum value of 1 unit and
 * a maximum value of 100 units.
 */
public abstract class Shape {
    
    /**
     * The minimum width and height.
     */
    public static final double MIN_LENGTH = 1.0;
    
    /**
     * The maximum width and height.
     */
    public static final double MAX_LENGTH = 100.0;
    
    protected double width;
    protected double height;
    
    /**
     * Initializes this shape to have a width and height of 1.
     */
    public Shape() {
        this.width = 1.0;
        this.height = 1.0;
    }
    
    /**
     * Returns the width of this shape. The returned width is greater
     * than or equal to 1.0.
     *
     * @return the width of this shape
     */
    public double width() {
        return this.width;
    }
    
    /**
     * Returns the height of this shape. The returned height is greater
     * than or equal to 1.0.
     *
     * @return the height of this shape
     */
    public double height() {
        return this.height;
    }
    
    /**
     * Changes both the width and height of this shape to the specified
     * dimensions. The width and height must be greater than or equal to 1.
     *
     * @param width the width of the shape
     * @param height the height of the shape
     * @throws IllegalArgumentException if either width or height or is out of range
     */
    public void setDimensions(double width, double height) {
        if (width < MIN_LENGTH || height < MIN_LENGTH) {
            throw new IllegalArgumentException("dimension too small");
        }
        if (width > MAX_LENGTH || height > MAX_LENGTH) {
            throw new IllegalArgumentException("dimension too large");
        }
        this.width = width;
        this.height = height;
    }
    
    /**
     * Returns an upper bound on the area of this shape. Some shapes have
     * complicated geometry and computing the exact area of such shapes is
     * difficult. This method returns an estimate of the area of a shape
     * where the estimate is guaranteed to be greater than or equal to
     * the true area of the shape.
     *
     * <p>
     * This method simply returns a value equal to the width times the
     * height of the shape. Subclasses should override this method if they can 
     * provide a tighter upper bound.
     *
     * @return an upper bound on the area of the shape
     */ 
    public double areaUpperBound() {
        return this.width * this.height;
    }
    
    // other methods not shown
}

In [ ]:
public class Rectangle extends Shape {
    
    public Rectangle() {
        super();
    }
}

A programmer uses the `Shape` hierarchy of classes to create a simple animation where various shapes move and change size slowly over time. The programmer creates the following method to change the size of a shape by a random small amount:

In [ ]:
public class ShapeUtils {
    
    public static void randomScale(Shape s) {
        // randomly scales the width and height of a shape by a small amount (less than 1%)
        // Math.random() returns a random value between 0 and 1
        double w = s.width() * (1.0 + (Math.random() * 0.01 - 0.005));
        double h = s.height() * (1.0 + (Math.random() * 0.01 - 0.005));
        s.setDimensions(w, h);
    }
}

In [ ]:
Shape s = new Rectangle();
s.setDimensions(10.0, 5.0);
System.out.println("(width, height) = (" + s.width() + ", " + s.height() + ")");
ShapeUtils.randomScale(s);
System.out.println("(width, height) = (" + s.width() + ", " + s.height() + ")");

After testing their method the programmer is confident that their method is correct *given the contracts of the `Shape` class and its methods*.

**Exercise 3** The programmer is actually incorrect; there are potential errors in their method. Can you identify them?

A second programmer creates a new shape class by subclassing the `Rectangle` class. Their thinking is something along the lines of "Mathematically, a square is a special kind of rectangle where the width and height are equal" so I can subclass `Rectangle` to create a `Square` class. If I override the `setDimensions` method I can ensure that the caller always uses the same values for the width and height:

In [ ]:
public class Square extends Rectangle {
    
    public Square() {
        super();
    }
    
    @Override
    public void setDimensions(double width, double height) {
        if (width < MIN_LENGTH || height < MIN_LENGTH) {
            throw new IllegalArgumentException("dimension too small");
        }
        if (width > MAX_LENGTH || height > MAX_LENGTH) {
            throw new IllegalArgumentException("dimension too large");
        }
        if (width != height) {                         // uh, oh, overriding method adds an extra constraint
            throw new IllegalArgumentException("width must be equal to height");
        }
        this.width = width;
        this.height = height;
    }
}

Notice that the `Square` class overrides the `setDimensions` method it inherits from `Shape` and adds the constraint that `width == height` must be `true`.  The problem with the `Square` implementation is that it breaks the `randomScale` method:

In [ ]:
Shape s = new Square();
s.setDimensions(5.0, 5.0);
System.out.println("(width, height) = (" + s.width() + ", " + s.height() + ")");
ShapeUtils.randomScale(s);   // exception here
System.out.println("(width, height) = (" + s.width() + ", " + s.height() + ")");

This example illustrates two important points to remember when working in an inheritance hierarchy.

1. Your intuition is often wrong when creating classes using inheritance. While it is true that a square is a type of rectangle in geometry, it is not automatically true that a `Square` is a `Rectangle` in a programming sense. In this example, the `Shape` class documents that a `Square` cannot be a `Rectangle`:

    ```java
    /**
     * Superclass for shapes that have an independent width and height.
     */
    public abstract class Shape
    ```
    
    The programmer has stated that the `Shape` class is an abstraction of a shape that has an independent width and height. A square does not have an independent width and height, and trying to fit a square class into this hierarchy is almost guaranteed to fail.
    
2. When a subclass fails to be substitutable for its superclasses any code written using the superclass contracts may fail. In this example, the failure of substitutability is caused by the `Square` programmer adding a new constraint to the the parameters of `setDimensions`.

A subclass should not add constraints to parameters when it overrides a method, but it is allowed to *remove or weaken* constraints. For example, the `Rectangle` class can override `setDimensions` so that negative values do not cause an exception to be thrown:

In [ ]:
public class Rectangle extends Shape {
    
    public Rectangle() {
        super();
    }
    
    /**
     * Changes both the width and height of this shape to the specified
     * dimensions. Widths or heights that are out of range are clamped
     * to Shape.MIN_LENGTH and Shape.MAX_LENGTH.
     *
     * @param width the width of the shape
     * @param height the height of the shape
     */
    @Override
    public void setDimensions(double width, double height) {
        // make sure width/height is at least MIN_LENGTH
        width = Math.max(width, MIN_LENGTH);
        height = Math.max(height, MIN_LENGTH);
        
        // make sure width/height is at most MAX_LENGTH
        width = Math.min(width, MAX_LENGTH);
        height = Math.min(height, MAX_LENGTH);
        
        this.width = width;
        this.height = height;
    }
}

The change in behaviour is acceptable because the `Rectangle` version of `setDimensions` behaves the same as the `Shape` version for all values that the `Shape` version considers to be acceptable.

## Method return values 

If a superclass method promises something about its return value, then a subclass that overrides the method must ensure that the overridden method returns a value that satisfies the promise of the superclass. If a subclass returns a value that does not satisfy the promise of the superclass then code created using the superclass contracts will fail when given a subclass object.

Recall that the `width` method of `Shape` promises to never return a value that is less than `1.0`:

```java
    // in Shape.java

    /**
     * Returns the width of this shape. The returned width is greater
     * than or equal to 1.0.
     *
     * @return the width of this shape
     */
    public double width() {
        return this.width;
    }
```

In many science and engineering problems that involve shape analysis, a common operation is to normalize a shape which involves uniformly scaling the shape so that its width or height (or area) is equal to 1. This is easy to accomplish *if the dimensions of the shape are not equal to zero*:

In [ ]:
public class ShapeAnalysis {
    
    public static void normalizeWidth(Shape s) {
        double h = s.height() / s.width();
        double w = 1.0;
        s.setDimensions(w, h);
    }
}

In [ ]:
Shape s = new Rectangle();
s.setDimensions(1.5, 4.2);
System.out.println("(width, height) = (" + s.width() + ", " + s.height() + ")");
ShapeAnalysis.normalizeWidth(s);
System.out.println("(width, height) = (" + s.width() + ", " + s.height() + ")");

After testing their method the programmer is confident that their method is correct *given the contracts of the `Shape` class and its methods*.

Suppose that a programmer creates a `VLine` class that represents a vertical line segment having zero width:

In [ ]:
/**
 * A shape representing a zero-width vertical line.
 */
public class VLine extends Shape {
    
    public VLine() {
        super();
        this.width = 0;     // uh, oh, width of zero
    }
    
    @Override
    public double width() { 
        return 0.0;
    }
}

Notice that the `VLine` class overrides the `width` method it inherits from `Shape` so that it always returns `0.0`. The problem with the `VLine` implementation is that it intentionally breaks one of the class invariants (it sets the width to be less than `1.0`) and it causes `normalizeWidth` to fail because the value of `h` in `double h = s.height() / s.width();` is equal to `Double.POSITIVE_INFINITY` and `setDimensions(w, h)` throws an exception when `h` is greater than `Shape.MAX_LENGTH`:

In [ ]:
Shape s = new VLine();
System.out.println("(width, height) = (" + s.width() + ", " + s.height() + ")");
ShapeAnalysis.normalizeWidth(s);
System.out.println("(width, height) = (" + s.width() + ", " + s.height() + ")");

A subclass is allowed to return a different value from an overridden method as long as the value satisfies the contract of the superclass. For example, a `Triangle` class could override `areaUpperBound` to return the true area of the triangle because the area of a triangle is simply one-half its width times its height:

In [ ]:
public class Triangle extends Shape {
    
    public Triangle() {
        super();
    }
    
    /**
     * Returns the true area of this triangle.
     *
     * @return the area of this triangle
     */ 
    @Override
    public double areaUpperBound() {
        return 0.5 * this.width * this.height;
    }
    
}

In [ ]:
Shape s = new Triangle();
System.out.println(s.areaUpperBound());

**Exercise 4** North American shoe sizes include half-sizes (e.g., size 8.5). The smallest adult shoe size is 1 although it is difficult to find shoes sizes less than 5 in most retail stores. Suppose that you have a `Shoe` class with a `size` method that promises to return the size of the shoe where the returned size is 1 or greater. Some specialty shoes only come in whole sizes. Can a `SpecialtyShoe` class override the `size` method and promise that it returns only whole sizes equal to 1 or greater?

**Exercise 5** The `Random` class has a method `nextInt` that promises to return a random integer value where the odds of returning each possible int value are equal. Can a subclass of `Random` override the `nextInt` method so that it returns random even integer values where the odds of returning each possible even int value are equal?

## Methods that change the state of an object

If a superclass method promises to change the state of an object in a particular way, then a subclass that overrides the method must ensure that the overridden method changes the state of an object in a way that satisfies the promise of the superclass. If a subclass changes the state of the object in a way that does not satisfy the promise of the superclass then code created using the superclass contracts will fail when given a subclass object.

Consider the following method that scales a rectangle so that it has the specified aspect ratio (the ratio of the shape's width to its height):

In [ ]:
public class RectangleUtil {
    
    public enum Options {
        KEEP_CURRENT_WIDTH,
        KEEP_CURRENT_HEIGHT
    }
    
    /**
     * Sets the dimensions of the specified rectangle so that it has the
     * specified aspect ratio. If changing the dimensions of the shape
     * causes the dimensions of the shape to become out of range then
     * this method leaves the dimensions of the shape unchanged and returns
     * false.
     * 
     * <p>
     * The opt parameter specifies whether the method should keep the current
     * width or the current height when changing the dimensions of the shape.
     *
     * @param r a rectangle
     * @param aspectRatio the desired aspect ratio
     * @param opt the option controlling which dimension should remain the same
     * @return true if the dimensions of the rectangle are successfully changed,
     *     false otherwise
     */
    public static boolean setAspectRatio(Rectangle r, double aspectRatio, Options opt) {
        double w = r.width();
        double h = r.height();
        if (opt == Options.KEEP_CURRENT_WIDTH) {
            h = w / aspectRatio;
            if (h < Shape.MIN_LENGTH || h > Shape.MAX_LENGTH) {
                return false;
            }
        }
        else {
            w = h * aspectRatio;
            if (w < Shape.MIN_LENGTH || w > Shape.MAX_LENGTH) {
                return false;
            }
        }
        r.setDimensions(w, h);
        return true;
    }
}

In [ ]:
Rectangle s = new Rectangle();

s.setDimensions(3.0, 2.0);
System.out.println("(width, height, aspect ratio) = (" + s.width() + ", " + s.height() + ", " + (s.width() / s.height()) + ")");

RectangleUtil.setAspectRatio(s, 2.0, RectangleUtil.Options.KEEP_CURRENT_WIDTH);
System.out.println("(width, height, aspect ratio) = (" + s.width() + ", " + s.height() + ", " + (s.width() / s.height()) + ")");

RectangleUtil.setAspectRatio(s, 5.0, RectangleUtil.Options.KEEP_CURRENT_HEIGHT);
System.out.println("(width, height, aspect ratio) = (" + s.width() + ", " + s.height() + ", " + (s.width() / s.height()) + ")");

Suppose that the `Square` programmer is determined to have `Square` extend `Rectangle` and they re-write their class like so:

In [ ]:
public class Square extends Rectangle {
    
    public Square() {
        super();
    }
    
    @Override
    public void setDimensions(double width, double height) {
        if (width < MIN_LENGTH || height < MIN_LENGTH) {
            throw new IllegalArgumentException("dimension too small");
        }
        if (width > MAX_LENGTH || height > MAX_LENGTH) {
            throw new IllegalArgumentException("dimension too large");
        }
        this.width = width;
        this.height = width;   // ignore height parameter
    }
}

The overridden `setDimensions` method ensures that the square always has an equal width and height by ignoring the `height` parameter however it cause the `setAspectRatio` method to fail to fulfill its contract because the width and height of the square cannot be changed independently:

In [ ]:
Rectangle s = new Square();

s.setDimensions(3.0, 2.0);
System.out.println("(width, height, aspect ratio) = (" + s.width() + ", " + s.height() + ", " + (s.width() / s.height()) + ")");

RectangleUtil.setAspectRatio(s, 2.0, RectangleUtil.Options.KEEP_CURRENT_WIDTH);
System.out.println("(width, height, aspect ratio) = (" + s.width() + ", " + s.height() + ", " + (s.width() / s.height()) + ")");   // oops, aspect ratio not 2

RectangleUtil.setAspectRatio(s, 5.0, RectangleUtil.Options.KEEP_CURRENT_HEIGHT);
System.out.println("(width, height, aspect ratio) = (" + s.width() + ", " + s.height() + ", " + (s.width() / s.height()) + ")");   // oops aspect ratio not 5

**Exercise 6** Suppose that the `Square` programmer is still determined to have `Square` extend `Rectangle` and they re-write their class like so:

In [ ]:
public class Square extends Rectangle {
    
    public Square() {
        super();
    }
    
    public void setLength(double length) {
        super.setDimensions(length, length);
    }
    
    @Override
    public void setDimensions(double width, double height) {
        // do nothing
    }
}

What problem does this solution have when trying to program using the `Shape` class contracts?

**Exercise 7** Recall the `AbstractAffinity` class from Assignment 4. Why does the class not have a method called `setMatrix` that allows the caller to set the matrix representing the transformation?

**Exerise 8** Can the `Affinity` class from Assignment 4 have a method called `setMatrix` that allows the caller to set the matrix representing the transformation?